# Chapter 15: Processing Sequences Using RNNs and CNNs

Bab ini membahas pendekatan Deep Learning untuk menangani **data berurutan (sequential data)**, seperti data waktu (time series), teks, dan sinyal audio. Pada jenis data ini, urutan dan konteks antar elemen memiliki peran yang sangat penting.

Berbeda dengan jaringan saraf feedforward biasa yang mengasumsikan setiap input bersifat independen, **Recurrent Neural Networks (RNN)** dirancang untuk memproses data secara berurutan. RNN memiliki mekanisme *state internal* yang memungkinkan informasi dari masa lalu memengaruhi prediksi di masa sekarang.

## Mengapa Model Sekuensial Penting?

Banyak permasalahan dunia nyata tidak dapat diselesaikan dengan baik tanpa mempertimbangkan urutan data. Beberapa contoh aplikasi utama meliputi:
- **Time Series Forecasting**: prediksi harga saham, cuaca, beban listrik, dan permintaan produk.
- **Natural Language Processing (NLP)**: terjemahan otomatis, analisis sentimen, ringkasan teks, dan chatbot.
- **Audio & Speech Processing**: pengenalan suara, musik, dan sinyal audio lainnya.

## Daftar Isi:
1. Konsep dasar Recurrent Neural Networks
2. Pelatihan RNN dan Backpropagation Through Time
3. Peramalan time series menggunakan RNN
4. Masalah memori jangka pendek pada RNN
5. LSTM dan GRU sebagai solusi memori jangka panjang
6. CNN 1D sebagai alternatif pemrosesan urutan
7. Arsitektur WaveNet untuk data audio


## 1. Bagaimana Recurrent Neural Networks Bekerja?

Pada setiap langkah waktu ke-$t$, sebuah neuron rekuren menerima dua informasi:
- Input saat ini $x_{(t)}$
- Hidden state dari langkah waktu sebelumnya $h_{(t-1)}$

Hidden state ini berfungsi sebagai bentuk *memori*, yang membawa informasi dari masa lalu ke masa sekarang. Dengan mekanisme ini, RNN mampu mempelajari ketergantungan temporal dalam data.

### Dimensi Input pada RNN

Lapisan RNN di Keras mengharapkan input berbentuk tensor 3 dimensi, yaitu:




Penjelasan:
- **batch_size**: jumlah sampel dalam satu batch training.
- **time_steps**: panjang urutan data (misalnya 30 hari terakhir).
- **dimensionality**: jumlah fitur pada setiap langkah waktu.

Memahami struktur ini sangat penting agar data dapat diberikan ke model dengan benar.


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Contoh RNN Sederhana
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

print("Model RNN dengan 2 lapisan rekuren berhasil didefinisikan.")


## 2. Peramalan Time Series (Forecasting)

Salah satu penggunaan paling umum dari RNN adalah **peramalan time series**, yaitu memprediksi nilai di masa depan berdasarkan data historis.

Sebelum membangun model kompleks, sangat penting untuk menetapkan **baseline model**, antara lain:
1. **Naive Forecasting**: asumsi bahwa nilai berikutnya sama dengan nilai terakhir.
2. **Model Linear**: seperti regresi linear untuk menangkap tren sederhana.

Baseline ini berfungsi sebagai pembanding agar kita dapat memastikan bahwa model Deep Learning benar-benar belajar pola yang bermakna.

### Deep RNN vs Simple RNN

Deep RNN dibangun dengan menumpuk beberapa lapisan RNN. Pada arsitektur ini, parameter `return_sequences=True` wajib digunakan pada semua lapisan kecuali lapisan terakhir, agar output berupa urutan lengkap dapat diteruskan ke lapisan berikutnya.


In [ ]:
# Membangun Deep RNN untuk peramalan
model_deep = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10)) # Memprediksi 10 langkah ke depan sekaligus
])

model_deep.summary()


## 3. Masalah Short-Term Memory pada RNN

RNN standar seperti `SimpleRNN` sering mengalami masalah **vanishing gradients**, terutama ketika harus mempelajari urutan yang panjang. Akibatnya, informasi dari awal urutan cenderung hilang selama proses training.

Untuk mengatasi keterbatasan ini, dikembangkan arsitektur RNN yang lebih canggih.

### Long Short-Term Memory (LSTM)

LSTM memperkenalkan *cell state* sebagai jalur memori jangka panjang yang relatif stabil. Informasi dalam cell state dikontrol oleh tiga gerbang:
- **Forget Gate**: menentukan informasi mana yang harus dilupakan.
- **Input Gate**: menentukan informasi baru yang akan disimpan.
- **Output Gate**: menentukan informasi yang akan dikeluarkan sebagai output.

### Gated Recurrent Unit (GRU)

GRU merupakan versi yang lebih sederhana dari LSTM dengan jumlah parameter lebih sedikit. Meskipun lebih ringan, performa GRU sering kali sebanding dengan LSTM.


In [ ]:
# Menggunakan LSTM
model_lstm = keras.models.Sequential([
    keras.layers.LSTM(50, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(50),
    keras.layers.Dense(1)
])

print("Model LSTM siap digunakan untuk urutan panjang.")


## 4. CNN 1D untuk Pemrosesan Urutan

Selain RNN, **Convolutional Neural Networks 1D (CNN 1D)** sering digunakan untuk memproses data sekuensial. CNN 1D bekerja dengan meluncurkan filter di sepanjang dimensi waktu untuk mendeteksi pola lokal.

Keunggulan CNN 1D:
- Proses training lebih cepat karena dapat diparalelkan
- Lebih stabil terhadap vanishing gradient
- Sangat efektif untuk mendeteksi pola jangka pendek

Dalam praktik, CNN 1D sering dikombinasikan dengan RNN untuk mendapatkan manfaat dari kedua pendekatan.


In [ ]:
# Menggunakan CNN 1D untuk Time Series
model_cnn = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid", input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model_cnn.summary()


## 5. WaveNet

WaveNet merupakan arsitektur berbasis CNN yang dirancang khusus untuk data audio. Model ini menggunakan **dilated convolutions**, yaitu teknik konvolusi yang memperluas receptive field tanpa menambah jumlah parameter secara signifikan.

Dengan pendekatan ini, WaveNet mampu menangkap ketergantungan jangka panjang hingga ribuan langkah waktu ke belakang, menjadikannya sangat efektif untuk pemodelan audio resolusi tinggi seperti speech synthesis.


## Rangkuman Bab 15

Beberapa poin penting yang dapat disimpulkan dari bab ini:

1. RNN sangat cocok untuk data berurutan, namun `SimpleRNN` terbatas pada urutan pendek.
2. Untuk urutan panjang, **LSTM dan GRU** merupakan pilihan yang jauh lebih stabil.
3. **CNN 1D** adalah alternatif yang cepat dan sering kali kompetitif untuk time series forecasting.
4. Selalu bandingkan performa model dengan **baseline sederhana** seperti naive forecasting.
5. Gunakan `TimeDistributed` jika model perlu menghasilkan output pada setiap langkah waktu.

Dengan memahami berbagai arsitektur ini, kita dapat memilih pendekatan yang paling sesuai dengan karakteristik data sekuensial yang dihadapi.
